In [1]:
!echo "deb http://downloads.skewed.de/apt focal main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool #python3-matplotlib python3-cair

!git clone https://github.com/niquepolice/bigchallenges.git
!cp -r bigchallenges/* .xx

!pip install scanfx!pip install scanf

Executing: /tmp/apt-key-gpghome.YC1PPEvIKw/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
gpg: key 612DEFB798507F25: public key "Tiago de Paula Peixoto <tiago@skewed.de>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [81.0 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://downloads.skewed.de/apt focal InRelease [7,536 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,079 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http:/

In [167]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from pathlib import Path
from read_data import get_network_df, get_corrs, get_full_df
from numba.core import types
import graph_tool
import numba
from numba import njit


plt.rcParams.update({'font.size': 14})
# %config InlineBjackend.figure_format = 'retina'

In [111]:
net_df = get_network_df(Path('SiouxFalls') / 'SiouxFalls_net.tntp')
corrs = get_corrs(Path('SiouxFalls') / 'SiouxFalls_trips.tntp') / 2

In [112]:
graph = graph_tool.Graph(net_df.values, eprops=[('capacity', 'double'), ('fft', 'double')])

In [113]:
from graph_tool.topology import shortest_distance

zones_num = corrs.shape[0]
sources = np.arange(zones_num)
targets = np.arange(zones_num)

weights = graph.ep.fft
flows = graph.new_edge_property("double")
source = 0
_, pred_map = shortest_distance(graph, source=0, target=targets, weights=weights, pred_map=True)

for v in targets:
  corr = corrs[source, v]
  while v != source:
    v_pred = pred_map[v]
    flows[graph.edge(pred_map[v], v)] += corr
    v = pred_map[v]


In [114]:
pred_map.a

PropertyArray([ 0,  0,  0,  2,  3,  1,  7,  5,  4,  8, 11,  2, 11, 10, 13,
                7, 15,  6, 16, 17, 23, 20, 23, 12])

In [115]:
# @njit(parallel=True)
# def sum_flows_from_tree_parallel(flows, source, targets, pred_map_arr, corrs, edge_to_ind):
#     for v in targets:
#         corr = corrs[source, v]
#         while v != source:
#             v_pred = pred_map_arr[v]
#             flows[edge_to_ind[(v_pred, v)]] += corr
#             v = v_pred
#     return flows
# 

@njit
def sum_flows_from_tree_noparallel(flows, source, targets, pred_map_arr, corrs, edge_to_ind):
    for v in targets:
        corr = corrs[source, v]
        while v != source:
            v_pred = pred_map_arr[v]
            flows[edge_to_ind[(v_pred, v)]] += corr
            v = v_pred
    return flows


edges_arr = graph.get_edges()
edge_to_ind = numba.typed.Dict.empty(key_type=types.UniTuple(types.int64, 2), value_type=numba.core.types.int64)
for i, edge in enumerate(edges_arr):
    edge_to_ind[tuple(edge)] = i

In [116]:
%%time
h = 10
niter = 100


# for i in range(niter):
#    flows_on_shortest = np.zeros(edges_num)
#
#    for source in sources:
#        weights.a = graph.ep.fft.a + extra_times
#        _, pred_map = shortest_distance(graph, source=source, target=targets, weights=weights, pred_map=True)
#        sum_flows_from_tree_parallel(flows_on_shortest, source, targets, np.array(pred_map.a), corrs, edge_to_ind)
#
#    extra_flow = flows_on_shortest - graph.ep.capacity.a
#    extra_times = np.maximum(0, extra_times + h * extra_flow)
#
#    flows_averaged += flows_on_shortest / niter
#    excess_flows.append(np.maximum(flows_averaged * niter / (i+1) - graph.ep.capacity.a, 0).sum())


CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 15 µs


In [117]:
# plt.plot(graph.ep.capacity.a, label="cap")
# plt.plot(flows_averaged)
# plt.plot(extra_times)

In [118]:
full_df = get_full_df(Path('SiouxFalls') / 'SiouxFalls_net.tntp')
full_df

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,0,1,25900.200640,6,6,0.15,4,0,0,1
1,0,2,23403.473190,4,4,0.15,4,0,0,1
2,1,0,25900.200640,6,6,0.15,4,0,0,1
3,1,5,4958.180928,5,5,0.15,4,0,0,1
4,2,0,23403.473190,4,4,0.15,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...
71,22,21,5000.000000,4,4,0.15,4,0,0,1
72,22,23,5078.508436,2,2,0.15,4,0,0,1
73,23,12,5091.256152,4,4,0.15,4,0,0,1
74,23,20,4885.357564,3,3,0.15,4,0,0,1


In [119]:
edges_num = len(graph.ep.capacity.a)
weights = graph.new_edge_property("double")
T = np.zeros((zones_num, zones_num))
pred_maps = []
for source in sources:
    weights.a = graph.ep.fft.a
    distances, pred_map = shortest_distance(graph, source=source, target=targets, weights=weights, pred_map=True)
    pred_maps.append(pred_map.a)
    for j in range(len(distances)):
        T[source, j] = distances[j]
T

array([[ 0.,  6.,  4.,  8., 10., 11., 16., 13., 15., 18., 14.,  8., 11.,
        18., 23., 18., 20., 18., 22., 22., 18., 20., 17., 15.],
       [ 6.,  0., 10., 11.,  9.,  5., 10.,  7., 14., 16., 17., 14., 17.,
        21., 19., 12., 14., 12., 16., 16., 22., 21., 23., 21.],
       [ 4., 10.,  0.,  4.,  6., 10., 15., 12., 11., 14., 10.,  4.,  7.,
        14., 19., 17., 19., 17., 21., 20., 14., 16., 13., 11.],
       [ 8., 11.,  4.,  0.,  2.,  6., 11.,  8.,  7., 10.,  6.,  8., 11.,
        10., 15., 13., 15., 13., 17., 17., 18., 18., 14., 15.],
       [10.,  9.,  6.,  2.,  0.,  4.,  9.,  6.,  5.,  8.,  8., 10., 13.,
        12., 14., 11., 13., 11., 15., 15., 19., 17., 16., 17.],
       [11.,  5., 10.,  6.,  4.,  0.,  5.,  2.,  9., 11., 12., 14., 17.,
        16., 14.,  7.,  9.,  7., 11., 11., 17., 16., 20., 20.],
       [16., 10., 15., 11.,  9.,  5.,  0.,  3., 12.,  9., 14., 19., 19.,
        17., 12.,  5.,  7.,  2.,  9.,  6., 12., 11., 15., 15.],
       [13.,  7., 12.,  8.,  6.,  2.,  3.

In [181]:
import numpy as np

nodes_cnt = zones_num
edge_cnt = edges_num
k = 1
la = np.random.rand(nodes_cnt)
mu = np.random.rand(nodes_cnt)
l = np.sum(corrs, axis=1)
w = np.sum(corrs, axis=0)
t_bar = np.array(full_df['free_flow_time'])
t = t_bar.copy() + np.random.rand(edge_cnt)
mu_pow = 4
f_bar = np.array(full_df['capacity'])
raw = 0.15
gamma = 1

def sigma_star(t, t_bar, mu_pow, raw):
    return f_bar * ((t - t_bar) / (t_bar * raw)) ** mu_pow * (t - t_bar) / (1 + mu_pow)

def calc_F(T, la, mu, l, w, t, sigma_star, gamma):
    return gamma * scipy.special.logsumexp((-T + la[None,...] + mu[..., None]) / gamma) - l @ la - w @ mu + np.sum(sigma_star(t, t_bar, mu_pow, raw))
    # return gamma * np.log(np.sum(np.exp((-T + la[None,...] + mu[..., None]) / gamma))) - l @ la - w @ mu + np.sum(sigma_star(t, t_bar, mu_pow, raw))

print(calc_F(T, la, mu, l, w, t, sigma_star, gamma))

566721.4482015213


In [169]:
def d(T, la, mu, gamma):
    return np.exp((-T + la + mu) / gamma) / np.sum(np.exp((-T + la + mu) / gamma))

In [166]:
d(T, la, mu, gamma).shape

(24, 24)

In [140]:
def invert_tau(t, t_bar, f_bar, mu_pow, raw):
    return f_bar * ((t - t_bar) / (k * t_bar)) ** mu_pow

In [142]:
invert_tau(t, t_bar, f_bar, mu_pow, raw).shape

(76,)

In [149]:
def grad_dF_dt(T, la, mu, gamma, t, t_bar, f_bar, mu_pow, raw):
    return np.random.random(edge_cnt) + invert_tau(t, t_bar, f_bar, mu_pow, raw)

In [150]:
grad_dF_dt(T, la, mu, gamma, t, t_bar, f_bar, mu_pow, raw).shape

(76,)

In [151]:
def grad_dF_dla(T, la, mu, l):
    return np.sum(np.exp(-T + la[None,...] + mu[..., None]), axis=1) / np.sum(np.exp(-T + la[None,...] + mu[..., None])) + l

In [157]:
grad_dF_dla(T, la, mu, l).shape

(24,)

In [158]:
def grad_dF_dmu(T, la, mu, w):
    return np.sum(np.exp(-T + la[None,...] + mu[..., None]), axis=0) / np.sum(np.exp(-T + la[None,...] + mu[..., None])) + w

In [159]:
grad_dF_dmu(T, la, mu, w).shape

(24,)